In [1]:
import nltk; 
import ssl
from pprint import pprint

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')

! pip install spacy
#python -m spacy download en_core_web_sm
#import spacy
#nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


[nltk_data] Downloading package stopwords to /Users/avtk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import re
import numpy as np
import pandas as pd


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
#import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [4]:
df_uk = pd.read_csv("../../data/processed/UK.csv")
df_us = pd.read_csv("../../data/processed/US.csv")
df_combined = pd.concat([df_uk,df_us])
print(df_combined.head(), df_combined.columns)
print(df_combined.columns)

                                         description  \
0  The strategy goes well beyond defeating an opp...   
1  All the conflict in Gaza is achieving is civil...   
2  Mexico has launched its army-run airline, with...   
3                           This blog is now closed.   
4  Palestinian Islamic Jihad posts videos of two ...   

                                            maintext  \
0  How to make sense of the sheer intensity of Is...   
1  All the conflict in Gaza is achieving is civil...   
2  For free real time breaking news alerts sent s...   
3  From 23 Dec 2023 18.49 CET UNRWA: 'People in G...   
4  The BBC will not be broadcasting the clip itse...   

               source_domain  \
0        www.theguardian.com   
1  www.unitedkingdomnews.net   
2      www.independent.co.uk   
3        www.theguardian.com   
4              www.bbc.co.uk   

                                               title  \
0  Israel’s use of disproportionate force is a lo...   
1  Israel cant defeat

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

words = list(sent_to_words(df_combined.title.values))

In [6]:
words

[['israel',
  'use',
  'of',
  'force',
  'is',
  'long',
  'established',
  'tactic',
  'with',
  'clear',
  'aim'],
 ['israel', 'cant', 'defeat', 'hamas', 'in', 'battle', 'so', 'what', 'next'],
 ['mexico',
  'army',
  'run',
  'airline',
  'takes',
  'to',
  'the',
  'skies',
  'with',
  'first',
  'flight',
  'to',
  'the',
  'resort',
  'of',
  'tulum'],
 ['death',
  'toll',
  'from',
  'israeli',
  'attacks',
  'tops',
  'as',
  'it',
  'happened'],
 ['israel', 'sees', 'sign', 'of', 'life', 'in', 'gaza', 'hostage', 'video'],
 ['putin',
  'left',
  'scrambling',
  'as',
  'russia',
  'monthly',
  'losses',
  'comparable',
  'to',
  'wwi'],
 ['us',
  'launches',
  'airstrikes',
  'on',
  'iran',
  'backed',
  'terrorists',
  'in',
  'iraq',
  'in',
  'retaliation',
  'for',
  'missile',
  'attack',
  'on',
  'american',
  'airbase',
  'that',
  'injured',
  'troops'],
 ['chinese',
  'performers',
  'bring',
  'cultural',
  'diversity',
  'to',
  'london',
  'lord',
  'mayor',
  'sho

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod)

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [9]:
data_words_nostops = remove_stopwords(words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [10]:
data_words_bigrams

[['israel', 'use', 'force', 'long', 'established', 'tactic', 'clear', 'aim'],
 ['israel', 'cant', 'defeat', 'hamas', 'battle', 'next'],
 ['mexico',
  'army',
  'run',
  'airline',
  'takes',
  'skies',
  'first',
  'flight',
  'resort',
  'tulum'],
 ['death_toll', 'israeli', 'attacks', 'tops', 'happened'],
 ['israel', 'sees', 'sign', 'life', 'gaza', 'hostage', 'video'],
 ['putin',
  'left',
  'scrambling',
  'russia',
  'monthly',
  'losses',
  'comparable',
  'wwi'],
 ['us',
  'launches',
  'airstrikes',
  'iran_backed',
  'terrorists',
  'iraq',
  'retaliation',
  'missile',
  'attack',
  'american',
  'airbase',
  'injured',
  'troops'],
 ['chinese',
  'performers',
  'bring',
  'cultural',
  'diversity',
  'london',
  'lord',
  'mayor',
  'show'],
 ['israel',
  'responds',
  'force',
  'support',
  'hamas',
  'soars',
  'west_bank',
  'october',
  'attack'],
 ['china',
  'issues',
  'furious',
  'rebuke',
  'biden',
  'branded',
  'president',
  'xi',
  'dictator',
  'hours',
  'pa

In [11]:
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]

In [12]:
# human readable format
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('aim', 1),
  ('clear', 1),
  ('established', 1),
  ('force', 1),
  ('israel', 1),
  ('long', 1),
  ('tactic', 1),
  ('use', 1)]]

In [13]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [14]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.049*"hostages" + 0.042*"china" + 0.036*"hospital" + 0.020*"pentagon" + '
  '0.016*"south" + 0.016*"un" + 0.015*"official" + 0.015*"time" + 0.014*"amid" '
  '+ 0.014*"talks"'),
 (1,
  '0.022*"day" + 0.020*"conflict" + 0.017*"kills" + 0.016*"christmas" + '
  '0.014*"ceasefire" + 0.014*"keep" + 0.014*"al_shifa" + 0.013*"launch" + '
  '0.012*"special" + 0.012*"calls"'),
 (2,
  '0.078*"us" + 0.062*"says" + 0.047*"military" + 0.029*"attack" + '
  '0.024*"troops" + 0.023*"new" + 0.021*"forces" + 0.020*"attacks" + '
  '0.017*"houthi" + 0.015*"iraq"'),
 (3,
  '0.078*"yemen" + 0.059*"strikes" + 0.025*"houthi_rebels" + 0.024*"red_sea" + '
  '0.023*"drones" + 0.021*"led" + 0.017*"linked" + 0.013*"announces" + '
  '0.013*"shipping" + 0.012*"coast"'),
 (4,
  '0.132*"israel" + 0.109*"gaza" + 0.095*"hamas" + 0.078*"war" + '
  '0.047*"israeli" + 0.014*"hostage" + 0.012*"fighting" + 0.010*"cease_fire" + '
  '0.009*"dead" + 0.007*"video"'),
 (5,
  '0.059*"iran" + 0.030*"netanyahu" + 0.030*"coul

In [15]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.090413809355207
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensi

In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')   
#vis

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.I

/Users/avtk/miniconda3/envs/bert/lib/python3.11/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.389430  0.199238       1        1  19.503028
4      0.108973  0.413823       2        1  18.088045
0     -0.300758 -0.228895       3        1   9.908290
9     -0.245141  0.322558       4        1   9.091014
8      0.169995 -0.364179       5        1   8.319887
1     -0.105252  0.135794       6        1   8.208617
7     -0.085245 -0.343455       7        1   8.071708
6     -0.376685  0.012381       8        1   7.822975
5      0.117045 -0.028196       9        1   6.264761
3      0.327637 -0.119070      10        1   4.721675, topic_info=          Term         Freq        Total Category  logprob  loglift
4       israel  1657.000000  1657.000000  Default  30.0000  30.0000
28        gaza  1372.000000  1372.000000  Default  29.0000  29.0000
101    ukraine   899.000000   899.000000  Default  28.0000  28.0000
11       hamas  1198.000000  1198.000000  Default  27.0000  27.0000
54          us  1056.000000  1056.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
519       used    25.427243    26.369263  Topic10  -4.8588   3.0166
2225  northern    24.199249    25.141198  Topic10  -4.9083   3.0148
1085   protect    23.157218    24.099309  Topic10  -4.9523   3.0131
2521     final    22.074854    23.016899  Topic10  -5.0001   3.0112
141    red_sea    79.360923   191.208150  Topic10  -3.7206   2.1736

[333 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
310       1  0.994541        aid
69        5  0.986977        air
1250      7  0.994695  air_force
289       5  0.981919   aircraft
1309     10  0.979666  airstrike
...     ...       ...        ...
1345      8  0.984726      years
423      10  0.998006      yemen
1075      3  0.976032        yet
379       8  0.989903   zelensky
400       8  0.988494  zelenskyy

[304 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 10, 9, 2, 8, 7, 6, 4])

In [17]:
data_words_nostops = remove_stopwords(words)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

id2word = corpora.Dictionary(data_words_trigrams)
texts = data_words_trigrams
corpus = [id2word.doc2bow(text) for text in texts]

In [18]:
# human readable format
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('aim', 1),
  ('clear', 1),
  ('established', 1),
  ('force', 1),
  ('israel', 1),
  ('long', 1),
  ('tactic', 1),
  ('use', 1)]]

In [19]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [20]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.047*"yemen" + 0.024*"pentagon" + 0.023*"weapons" + 0.021*"group" + '
  '0.020*"two" + 0.019*"shot" + 0.018*"week" + 0.018*"near" + 0.017*"drones" + '
  '0.016*"one"'),
 (1,
  '0.049*"houthi" + 0.040*"red_sea" + 0.032*"ship" + 0.030*"missiles" + '
  '0.027*"navy" + 0.020*"uk" + 0.020*"official" + 0.017*"zelenskyy" + '
  '0.016*"zelensky" + 0.014*"officials"'),
 (2,
  '0.137*"us" + 0.068*"biden" + 0.051*"attack" + 0.042*"troops" + '
  '0.034*"attacks" + 0.027*"iraq" + 0.019*"trump" + 0.012*"cease_fire" + '
  '0.011*"middle_east" + 0.011*"rocket"'),
 (3,
  '0.093*"russian" + 0.032*"ukrainian" + 0.030*"putin" + 0.024*"border" + '
  '0.021*"hit" + 0.020*"christmas" + 0.017*"market" + 0.014*"claims" + '
  '0.014*"kill" + 0.014*"billion"'),
 (4,
  '0.096*"ukraine" + 0.066*"russia" + 0.063*"military" + 0.029*"china" + '
  '0.018*"defense" + 0.017*"aid" + 0.012*"kyiv" + 0.012*"live_updates" + '
  '0.011*"report" + 0.008*"allies"'),
 (5,
  '0.082*"hostages" + 0.033*"palestinian" + 0.03

In [21]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_trigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.131267170247964
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensi

In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')   

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/avtk/miniconda3/envs/bert/lib/python3.11/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.397963  0.214125       1        1  26.713480
4      0.097316  0.390345       2        1  14.147813
6     -0.317525 -0.272458       3        1  11.751814
2     -0.255277  0.328935       4        1  10.443790
0      0.121794 -0.388698       5        1   7.268605
9     -0.099643  0.125504       6        1   6.415709
3     -0.062022 -0.242529       7        1   6.118025
1     -0.353437  0.011320       8        1   5.976149
7      0.143455 -0.028955       9        1   5.703583
5      0.327375 -0.137590      10        1   5.461032, topic_info=              Term         Freq        Total Category  logprob  loglift
4           israel  1717.000000  1717.000000  Default  30.0000  30.0000
54              us   985.000000   985.000000  Default  29.0000  29.0000
28            gaza  1390.000000  1390.000000  Default  28.0000  28.0000
101        ukraine   931.000000   931.000000  Default  27.0000  27.0000
11           hamas  1242.000000  1242.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
2413          jews    25.633922    26.551532  Topic10  -4.9852   2.8724
1572  south_africa    23.756809    24.674432  Topic10  -5.0613   2.8696
483          comes    22.875965    23.794493  Topic10  -5.0991   2.8682
428             eu    21.948534    22.866172  Topic10  -5.1404   2.8666
1227        losing    21.379796    22.297413  Topic10  -5.1667   2.8655

[333 rows x 6 columns], token_table=      Topic      Freq                 Term
term                                      
1768      5  0.969456               action
309       2  0.993588                  aid
69        2  0.984747                  air
1251      3  0.990279            air_force
288       5  0.969905             aircraft
...     ...       ...                  ...
424       5  0.996249                yemen
509       2  0.992925  yemen_houthi_rebels
1075      5  0.979481                  yet
380       8  0.989975             zelensky
401       8  0.983925            zelenskyy

[307 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 5, 7, 3, 1, 10, 4, 2, 8, 6])